In [ ]:
from avcv.all import *

In [ ]:
class Label:
    def __init__(self, label, video):
        self.label = label
        self.video = video
        
    def check_action_at_frame_idx(self, i):
        if isinstance(i, int):
            i = i/len(self.video)
            
        actions = []
        for action in self.label['annotation']['actionAnnotationList']:
            if i >= action['start'] and i < action['end']:
                action_idx = action['action']
                action_name = self.actionid2name[action_idx]
                actions.append(action_name)
        return actions
    @property
    def actionid2name(self):
        ret = dict()
        for actionLabel in self.label['config']['actionLabelData']:
            ret[actionLabel['id']] = actionLabel['name']
        return ret

In [ ]:
# ls /home/anhvth8/gitprojects/YOLOX/.cache/out_jsons

In [ ]:
def get_data(video_name):
    pred_json_path = f'/home/anhvth8/gitprojects/YOLOX/.cache/raw_video_predict_face_food/{video_name}/annotations/pred_mb2_face_food.json'
    root_video_name = '_'.join(video_name.split('_')[:-2])
    label_path = f'/data/DMS_Behavior_Detection/RawVideos/Action_Eating/{root_video_name}/{video_name}.json'
    video_path = f'/data/DMS_Behavior_Detection/RawVideos/Action_Eating/{root_video_name}/{video_name}.mp4'
    assert osp.exists(pred_json_path), f'404 {pred_json_path}'
    assert osp.exists(label_path), f'404 {label_path}'
    assert osp.exists(video_path), f'404 {video_path}'
    
    return dict(
        pred_json_path=pred_json_path,
        label_path = label_path,
        video_path=video_path
    )

In [ ]:
video_names  = os.listdir('/home/anhvth8/gitprojects/YOLOX/.cache/raw_video_predict_face_food/')
len(video_names)

83

In [ ]:
list_json_paths = []
for vname in video_names:
    try:
        json_paths = get_data(vname)
        list_json_paths.append(json_paths)
    except Exception as e:
        # print(e)
        pass

In [ ]:
len(list_json_paths)

42

In [ ]:
# label.label['config']['actionLabelData']

In [ ]:
!mkdir -p vis
for json_paths in tqdm(list_json_paths):
    v = mmcv.VideoReader(json_paths['video_path'])
    vis = []
    t = len(v)
    label = Label(mmcv.load(json_paths['label_path']), v)
    for i, frame in enumerate(v):
        actions = label.check_action_at_frame_idx(i)
        if len(actions):
            put_text(frame, (0, 100), f'Action: {actions[0]}', color=(0, 255, 0))
        vis.append(frame)
    vname = get_name(json_paths['video_path'])
    images_to_video(vis, f'vis/{vname}.mp4', output_size=(600, 400), verbose=1)
    break

  0%|                                                                                                  | 0/42 [00:00<?, ?it/s]2022-09-19 06:19:42.623 | INFO     | avcv.utils:images_to_video:267 - Write video, output_size: (600, 400)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 15/15, 230.2 task/s, elapsed: 0s, ETA:     0s

2022-09-19 06:19:42.691 | INFO     | avcv.utils:images_to_video:277 - -> /home/anhvth8/gitprojects/YOLOX/vis/hungng_Sensing_Session0_CAMc_1b_2c_3a_4c_5b_6b_7b_8a_9a_10a_11b_12a_13a_14b_15b_16f_17a_18a_19a_20b_eating_0014.mp4
  0%|                                                                                                  | 0/42 [00:00<?, ?it/s]


In [ ]:
# label.label